# test yolo-v5

## test dataloader

In [ ]:
import os, sys
sys.path.append(os.path.dirname(os.path.dirname(os.path.abspath('.'))))
sys.path.append(os.path.dirname(os.path.abspath('.')))
import pl_yolo_v5.src.dataset.dataset as Data
from imp import reload
reload(Data)

os.system(f'rm /data/ylw/code/yolo/yolov5-pytorch/test/test_rets/*.jpg')

In [ ]:
from torch.utils.data import DataLoader
from src.utils.voc_utils import get_anchors
from PyUtils.logs.print import *
from PyUtils.viz.plot_draw import heatmap
from src.dataset.dataset import YoloDataset
import numpy as np
import cv2


train_annotation_path = '../data/2007_train.txt'
with open(train_annotation_path, encoding='utf-8') as f:
    lines = f.readlines()
lines = lines[:120]

input_shape     = [640, 640]
num_classes = 20
anchors_path    = '../model_data/yolo_anchors.txt'
anchors, num_anchors     = get_anchors(anchors_path)
UnFreeze_Epoch      = 300
batch_size = 8
num_workers = 1
anchors_mask    = [[6, 7, 8], [3, 4, 5], [0, 1, 2]]
mosaic              = True
mosaic_prob         = 0.5
mixup               = True
mixup_prob          = 0.5

test_dataset   = YoloDataset(
    lines, 
    input_shape, 
    num_classes, 
    anchors, 
    anchors_mask, 
    mosaic=mosaic, 
    mixup=mixup, 
    mosaic_prob=mosaic_prob, 
    mixup_prob=mixup_prob, 
    train=True, 
)
loader = DataLoader(
    test_dataset, 
    shuffle = False, 
    batch_size = batch_size, 
    num_workers = num_workers, 
    pin_memory=True,
    drop_last=True, 
    collate_fn=test_dataset.yolo_dataset_collate
)

In [ ]:
for batch_idx, batch in enumerate(loader):
    sllog << f'[============][{batch_idx}]  {batch[0].shape=}  {len(batch[1])=} / {type(batch[1][0])=} / {batch[1][0].shape=}  {batch[2][0].shape=}  {batch[2][1].shape=}  {batch[2][2].shape=}'

    images = batch[0]
    for idx, image in enumerate(images):
        img = (image.numpy().transpose((1,2,0))*255).astype(np.uint8)
        img = cv2.UMat(img)
        
        for box in batch[1][idx].numpy():
            input_shape = img.get().shape[1]
            if len(box.shape) == 1:
                box = box[np.newaxis, :]
            bbox = box[:, :4] * input_shape
            cls = box[:, 4]
            for b in bbox:
                img = cv2.rectangle(
                    cv2.UMat(img), 
                    (int(b[0]-b[2]/2), int(b[1]-b[3]/2)), (int(b[0]+b[2]/2), int(b[1]+b[3]/2)), 
                    color=(0, 0, 255), thickness=2
                )

        iw_ret = cv2.imwrite(f'/data/ylw/code/pl_yolo_v5/test/rets2/{batch_idx}-{idx}.jpg', img)
        
    if (batch_idx+1) % 2 == 0:
        break;


In [ ]:
for batch_idx, batch in enumerate(loader):
    sllog << f'[============][{batch_idx}]  {batch[0].shape=}  {len(batch[1])=} / {batch[1][0].shape=}  {batch[2][0].shape=}  {batch[2][1].shape=}  {batch[2][2].shape=}'

    images = batch[0]
    for idx, image in enumerate(images):
        img = (image.numpy().transpose((1,2,0))*255).astype(np.uint8)
        # img = cv2.UMat(img)
        input_shape = img.shape[1]
        
        data1 = batch[2][0][idx, :, :, :, :].numpy().sum(axis=3).transpose((1,2,0))
        data2 = batch[2][1][idx, :, :, :, :].numpy().sum(axis=3).transpose((1,2,0))
        data3 = batch[2][2][idx, :, :, :, :].numpy().sum(axis=3).transpose((1,2,0))
        sllog << f'{data1.shape=}  {data2.shape=}  {data3.shape=}'
        data1 = cv2.resize(data1, (input_shape, input_shape))
        data2 = cv2.resize(data2, (input_shape, input_shape))
        data3 = cv2.resize(data3, (input_shape, input_shape))
        data = (data1 + data2 + data3).astype(np.uint8)
        
        # fig, ax = heatmap(data, save_file=f'/data/ylw/code/pl_yolo_v5/test/rets3/{batch_idx}-{idx}-hm.jpg', show=False)
        
        for box in batch[1][idx].numpy():
            if len(box.shape) == 1:
                box = box[np.newaxis, :]
            bbox = box[:, :4] * input_shape
            cls = box[:, 4]
            for b in bbox:
                img = cv2.rectangle(
                    cv2.UMat(img), 
                    (int(b[0]-b[2]/2), int(b[1]-b[3]/2)), (int(b[0]+b[2]/2), int(b[1]+b[3]/2)), 
                    color=(0, 0, 255), thickness=2
                )

        # iw_ret = cv2.imwrite(f'/data/ylw/code/pl_yolo_v5/test/rets3/{batch_idx}-{idx}-1.jpg', data1)
        # iw_ret = cv2.imwrite(f'/data/ylw/code/pl_yolo_v5/test/rets3/{batch_idx}-{idx}-2.jpg', data2)
        # iw_ret = cv2.imwrite(f'/data/ylw/code/pl_yolo_v5/test/rets3/{batch_idx}-{idx}-3.jpg', data3)
        # iw_ret = cv2.imwrite(f'/data/ylw/code/pl_yolo_v5/test/rets3/{batch_idx}-{idx}.jpg', data)
        img = cv2.addWeighted(cv2.UMat(img), 0.5, data, 0.8, 1)
        iw_ret = cv2.imwrite(f'/data/ylw/code/pl_yolo_v5/test/rets3/{batch_idx}-{idx}.jpg', img)
    
    if (batch_idx+1) % 2 == 0:
        break;


In [ ]:
import PyUtils.viz.plot_draw as draw
import numpy as np
a = np.linspace(0,99, 100).reshape((10, 10))
draw.heatmap(a, save_file='./hello.png', show=False);

## test model

In [ ]:
import os, sys
sys.path.append(os.path.dirname(os.path.dirname(os.path.abspath('.'))))
from src.nets.yolo import YoloBody
import torch
import numpy as np

In [ ]:
phi             = 's'
backbone        = 'cspdarknet'
num_classes = 20
anchors_path    = '../model_data/yolo_anchors.txt'
anchors, num_anchors     = get_anchors(anchors_path)
anchors_mask    = [[6, 7, 8], [3, 4, 5], [0, 1, 2]]
input_shape     = [640, 640]
pretrained      = False

model = YoloBody(anchors_mask, num_classes, phi, backbone, pretrained=pretrained, input_shape=input_shape)

In [ ]:
multi_scales = [320, 416, 640, 1024]  # [32, 16, 8]
for scale in multi_scales:
    input_data = torch.rand(size=(1, 3, scale, scale))
    preds = model(input_data)
    print(f'{preds[0].shape=}  {preds[1].shape=}  {preds[2].shape=}')


## test loss

In [ ]:
import src.loss.yolov5_loss as YoloLoss
reload(YoloLoss)

In [ ]:
num_workers = 4
batch_size = 4

test_dataset   = YoloDataset(
    lines, 
    input_shape, 
    num_classes, 
    anchors, 
    anchors_mask, 
    mosaic=mosaic, 
    mixup=mixup, 
    mosaic_prob=mosaic_prob, 
    mixup_prob=mixup_prob, 
    train=True, 
    fixed_input_shape=True
)
loader = DataLoader(
    test_dataset, 
    shuffle = False, 
    batch_size = batch_size, 
    num_workers = num_workers, 
    pin_memory=True,
    drop_last=True, 
    collate_fn=test_dataset.yolo_dataset_collate
)

In [ ]:
label_smoothing = 0
Cuda = False

Loss = YoloLoss.YOLOLoss(
    anchors, 
    num_classes, 
    input_shape, 
    anchors_mask, 
    label_smoothing
)



In [ ]:
for i in range(2):
    # 一个epoch内无法修改dataloader中input的尺寸
    for batch_idx, batch in enumerate(loader):
        data, bboxes, gts = batch
        sllog << f'[************][{i}][{batch_idx}]  {data.shape=}  {len(bboxes)=}  {gts[0].shape=}  {gts[1].shape=}  {gts[2].shape=}'
        preds = model(data)
        sllog << f'[{i}][{batch_idx}]  {preds[0].shape=}  {preds[1].shape=}  {preds[2].shape=}  {test_dataset.input_shape=}  {loader.dataset.input_shape=}'

        loss = 0
        with torch.no_grad():
            for idx, pred in enumerate(preds):
                loss += Loss(idx, pred, bboxes, gts[idx], batch)
        sllog << f'[{i}][{batch_idx}]  {loss.item()=}'
        
        input_scale = np.array([160, 160]) * (batch_idx+1)
        sllog << f'[============][{i}][{batch_idx}]  set input shape {input_scale}  {id(test_dataset)}  {id(loader)}  {id(test_dataset.input_shape)}  ...'
        # loader.dataset.input_shape = input_scale
        test_dataset.set_input_shape(160 * (batch_idx+1))
        sllog << f'='*80
        if (batch_idx+1) % 3 == 0:
            break


In [ ]:
16 / 8 * 2 / 1

In [ ]:
from turtle import shape
import numpy as np

a = np.array([[1,2,3,4], [5,6,7,8]])
b = np.array([[1,3,5,7]])
print(f'{a.shape=}  {b.shape=}')
c = np.concatenate((a, b), axis=0)
print(f'{c.shape=}')
d = np.array([])
print(f'{d.shape=}')
# e = np.concatenate((c, d), axis=0)
# print(f'{e.shape=}')
# f = np.append(d, a, axis=0)
# print(f'{f.shape=}')
g = np.vstack((a, b))
print(f'{g.shape=}')

In [ ]:
a = np.array([1,2,3])
print(f'[1]  {id(a)=}')
b = 1.0
a = a * b
print(f'[2]  {id(a)=}')

In [ ]:
a = [1,2]
print(a[len(a)])

In [ ]:
a = {}
print(len(a))
b = {'1':1, '2':[2,3,4]}
print(b)
a.update(b)
print(a)

In [ ]:
import torch
import torchvision as tv

In [ ]:
tv.models.MobileNetV2()

In [ ]:
import torch
from torch import nn
import torch.nn.functional as F

In [ ]:
class Head(nn.Module):
    def __init__(self,inp_c,out_c):
        super(Head,self).__init__()
        self.conv2d=nn.Conv2d(inp_c,out_c,3,2,padding=1,bias=False)
        self.bn=nn.BatchNorm2d(out_c)
        self.relu6=nn.ReLU6(inplace=True)
    
    def forward(self,x):
        x=self.conv2d(x)
        x=self.bn(x)
        out=self.relu6(x)
        return out

In [ ]:
class InvertResidual(nn.Module):
    def __init__(self,inp_c,out_c,stride,expand):
        super(InvertResidual,self).__init__()
        self.stride=stride
        out_ce=out_c*expand
        self.conv1_1=nn.Conv2d(inp_c,out_ce,1,bias=False)
        self.conv1_2=nn.Conv2d(out_ce,out_c,1,bias=False)
        self.bn1=nn.BatchNorm2d(out_ce)
        self.bn2=nn.BatchNorm2d(out_c)
        self.deepwide=nn.Conv2d(out_ce,out_ce,3,stride,padding=1,groups=out_c)
        self.relu6=nn.ReLU6(inplace=True)
        self.shortcut = nn.Sequential()
        if stride == 1 and inp_c != out_c:
            self.shortcut = nn.Sequential(
                nn.Conv2d(inp_c,out_c,1,bias=False),
                nn.BatchNorm2d(out_c))
            
        
    def forward(self,x):
        ori_x=x
        x=self.conv1_1(x)
        x=self.bn1(x)
        x=self.relu6(x)
        x=self.deepwide(x)
        x=self.bn1(x)
        x=self.relu6(x)
        x=self.conv1_2(x)
        y=self.bn2(x)
        out=y + self.shortcut(ori_x) if self.stride==1 else y
        return out

In [ ]:
class MobileNetv2(nn.Module):
    def __init__(self,width_mult=1,num_classes=100):
        super(MobileNetv2,self).__init__()
        block = InvertResidual
        input_channel = 32
        interverted_residual_setting = [
            # t, c, n, s
            [1, 16, 1, 1],
            [6, 24, 2, 2],
            [6, 32, 3, 2],
            [6, 64, 4, 2],
            [6, 96, 3, 1],
            [6, 160, 3, 2],
            [6, 320, 1, 1]]
        input_channel=int(input_channel * width_mult)
        head_layer=Head(3,input_channel)
        self.layers=[head_layer]
        for t, c, n, s in interverted_residual_setting:
            stride = s
            output_channel = int(c * width_mult)
            for i in range(n):
                if i==0:
                    self.layers.append(block(input_channel,output_channel,stride,t))
                else:
                    self.layers.append(block(input_channel,output_channel,1,t))
                input_channel=output_channel
                
        self.layers = nn.Sequential(*self.layers)
 
        self.conv_end = nn.Conv2d(320,1280, kernel_size=1, stride=1,padding=0, bias=False)
        self.bn_end = nn.BatchNorm2d(1280)
        self.relu=nn.ReLU6(inplace=True)
        self.AdaptiveAvgPool = nn.AdaptiveAvgPool2d((1, 1))
        self.linear = nn.Linear(1280, num_classes)
    def forward(self,x):
        x=self.layers(x)
        x=self.conv_end(x)
        x=self.bn_end(x)
        x=self.relu(x)
        x=self.AdaptiveAvgPool(x)
        x= x.view(x.size(0), -1)
        out = self.linear(x)
        return out

In [ ]:
if __name__=="__main__":
    test_input=torch.rand(1, 3, 480, 640)
    print(test_input.size())
    model= MobileNetv2()
    out=model(test_input )
    print(out.size())

In [ ]:
import time
for i in range(0, 101, 2):
    time.sleep(0.1)  #线程推迟指定时间运行，0.1秒代表休眠100毫秒
    num = i // 2
    if i == 100:
        process = "\r[%3s%%]: |%-50s|\n" % (i, '|' * num)
    else:
        process = "\r[%3s%%]: |%-50s|" % (i, '|' * num)
    print(process, end='', flush=True)

In [ ]:
import sys, time
print("正在下载......")
for i in range(11):#通过for循环输出进度条效果
    if i != 10:
        sys.stdout.write("==")
    else:
        sys.stdout.write("== " + str(i*10)+"%/100%")
        sys.stdout.flush()
    time.sleep(0.5)#sleep用来控制输出时间
print(" " + "下载完成")

In [ ]:
from time import sleep
from tqdm import tqdm
for i in tqdm(range(20)):#也可改写为for i in trange(20):
	sleep(0.5)#sleep用来控制输出时间

In [ ]:
import time
from progressbar import *
progress = ProgressBar()
for i in progress(range(1000)):
	time.sleep(0.01)#sleep用来控制输出时间

In [ ]:
for i in range(10):
    time.sleep(1)
    print(f'\r**== {i} ==**', end='', flush=True)
    

In [ ]:
import numpy as np
a = np.linspace(0, 9, 10).reshape((2,5))
for i in range(10):
    time.sleep(1)
    print(f'\r**== {list(a+i)} ==**', end='', flush=True)
    

In [ ]:
from prettytable import PrettyTable


data = [
    ["小明","01","男"],
    ["小红","02","女"],
    ["小黄","03","男"]
]

import numpy as np
a = np.linspace(0, 9, 10).reshape((2,5))
for i in range(5):
    time.sleep(1)
    table=PrettyTable(["姓名","学号","性别"])
    table.add_row(data[i%3])
    # print(f'\r{table.get_string()[:-1]}', end='', flush=True)
    sys.stdout.write(f'{table.get_string()[:-1]}')
    # table.clear()

In [ ]:
import random
import time

a = 0
for i in range(5):
    statement = """
    Line {}
    Line {}
    Line {}
    Value = {}
    """.format(random.random(), random.random(), random.random(), a)
    print(statement, end='\r')
    time.sleep(1)
    a += 1

In [ ]:
#!/usr/bin/python3

import curses
from time import sleep
from random import random

statement = """
Line {}
Line {}
Line {}
Value = {}"""

screen = curses.initscr()
n = 0

while n < 5:
    screen.clear()
    screen.addstr(0, 0, statement.format(random(), random(), random(), n))
    screen.refresh()
    n += 1
    sleep(0.5)

curses.endwin()

### 好用

In [ ]:
# 好用


import sys
import time
import os
import re
from easydl import clear_output
from prettytable import PrettyTable


data = [
    ["小明","01","男"],
    ["小红","02","女"],
    ["小黄","03","男"],
    ["小黄","04","男"],
    ["小黄","05","男"],
    ["小黄","06","男"]
]


# for i in range(3):
#     table=PrettyTable(["姓名","学号","性别"])
#     table.add_row(data[i%3])

#     os.system('clear')
#     clear_output()
#     sys.stdout.write("\r{0}".format(table.get_string()))
#     sys.stdout.flush() 
#     time.sleep(1)

table=PrettyTable(["姓名","学号","性别"])
table.add_row(data[0])
table.add_row(data[1])
table.add_row(data[2])
table.add_row(data[3])
table.add_row(data[4])
table.add_row(data[5])
print(f'{table.get_string()}  {len(table.rows)}  {type(table.rows)}')
# table.del_row(-1)
# print(f'[1]  {table.get_string()}  {len(table.rows)}  {type(table.rows)}')
# table.del_row(0)
# print(f'[2]  {table.get_string()}  {len(table.rows)}  {type(table.rows)}')
# table.clear_rows()
# print(f'[3]  {table.get_string()}  {len(table.rows)}  {type(table.rows)}')
keep = 20
table = table[-keep:]
print(f'[3]  {table.get_string()}  {len(table.rows)}  {type(table.rows)}')


In [ ]:

import time
import logging
import progressbar
 
# progressbar.streams.wrap_stderr()
# progressbar.StreamWrapper()
progressbar.utils.StreamWrapper()
logging.basicConfig()
 
for i in progressbar.progressbar(range(10)):
    logging.error('Got %d', i)
    time.sleep(0.2)

In [ ]:
bar = progressbar.ProgressBar(redirect_stdout=True)

for i in range(10):

    print('Some text', i)

    time.sleep(0.1)

    bar.update((i+1)*10)

In [ ]:
from tqdm import tqdm
import time

for i in tqdm(range(20), ascii=True,desc="1st loop"):
  for j in tqdm(range(10), ascii=True,desc="2nd loop"):
    time.sleep(0.01)

In [ ]:
with tqdm(total=100) as pbar:
    for i in range(10):
        time.sleep(0.1)
        pbar.update(10)

'''method 2'''
pbar = tqdm(total=100)
for i in range(10):
    time.sleep(0.1)
    pbar.update(10)
pbar.close()

In [17]:
# from tqdm import tqdm_notebook
import time
from tqdm.notebook import tqdm as tqdm_notebook

for i in tqdm_notebook(range(100),desc='demo：'):
    time.sleep(0.01)

demo：:   0%|          | 0/100 [00:00<?, ?it/s]

### 好

refs:

https://www.cnblogs.com/softlin/p/13339766.html



进度条默认格式为： {l_bar}{bar}{r_bar}

进度条分为三部分： 中间的图形(bar)，图形左边(l_bar)、图形右边(r_bar)

l_bar： {desc}: {percentage:3.0f}%|

bar: 进度条

r_bar: |{n_fmt}/{total_fmt}[{elapsed}<{remaining},{rate_fmt}{postfix}]

100%|█████████████████| 3/3 [00:03<00:00, 1.00s/it]


In [20]:
import numpy as np
from torch.utils.data import DataLoader
import time
from tqdm import tqdm, tqdm_notebook
from random import random

data =np.linspace(0, 9, 10)
data_loader = DataLoader(data, batch_size=2, num_workers=0, shuffle=False)

iterator = tqdm(
    data_loader,
    maxinterval=10,
    mininterval=2, 
    ncols=80,
    bar_format='{l_bar}|{bar}| {n_fmt}/{total_fmt} [{rate_fmt}{postfix}|{elapsed}<{remaining}]',
    nrows=10,smoothing=0.1)
epoch =0
for d in iterator:
    time.sleep(2)
    epoch +=1
    # print(d)
    iterator.set_description('epoch %d' %epoch)
    iterator.set_postfix_str('loss={:^7.3f}'.format(random()))



epoch 5: 100%||██████████████████████| 5/5 [ 2.01s/it, loss= 0.227 |00:10<00:00]


In [25]:
import logging
from tqdm import trange
from tqdm import tqdm, tqdm_notebook
from tqdm.contrib.logging import logging_redirect_tqdm
import time

LOG = logging.getLogger(__name__)

if __name__ == '__main__':
    logging.basicConfig(level=logging.INFO)
    with logging_redirect_tqdm():
        for i in trange(9):
            if i%2 == 0:
                LOG.info("console logging redirected to `tqdm.write()`")
            time.sleep(0.5)
    # logging restored


INFO:__main__:console logging redirected to `tqdm.write()`
  0%|          | 0/9 [00:00<?, ?it/s]

console logging redirected to `tqdm.write()`
console logging redirected to `tqdm.write()`


INFO:__main__:console logging redirected to `tqdm.write()`
 22%|██▏       | 2/9 [00:01<00:03,  1.98it/s]

console logging redirected to `tqdm.write()`
console logging redirected to `tqdm.write()`


INFO:__main__:console logging redirected to `tqdm.write()`
 44%|████▍     | 4/9 [00:02<00:02,  1.97it/s]

...

INFO:__main__:console logging redirected to `tqdm.write()`
 67%|██████▋   | 6/9 [00:03<00:01,  1.97it/s]

console logging redirected to `tqdm.write()`
console logging redirected to `tqdm.write()`


INFO:__main__:console logging redirected to `tqdm.write()`
 89%|████████▉ | 8/9 [00:04<00:00,  1.97it/s]

console logging redirected to `tqdm.write()`
console logging redirected to `tqdm.write()`


100%|██████████| 9/9 [00:04<00:00,  1.97it/s]


In [33]:
a = {'a':1, 'b':2}
list(a.keys())
list(a.values())
b = {'c':3, 'd':4}
a.update(b)
a

['a', 'b']

[1, 2]

{'a': 1, 'b': 2, 'c': 3, 'd': 4}

In [30]:
a = [1,2,3]
b=[4,5,6]
a.extend(b)
a

[1, 2, 3, 4, 5, 6]

In [1]:
a = [2,3,4,5,6,7,8,9,0]
xyz = [0,12,4,6,242,7,9]
print([x for x in xyz if x in a else 'inf'])

SyntaxError: invalid syntax (1615935208.py, line 3)